In [1]:
import pandas as pd

In [2]:
df = pd.read_feather('../data/train_dataset.fth')
df.head()

,index,id,cell_id,cell_type,source,rank,pct_rank,ancestor_id,parent_id
0,0,00001756c60be8,1862f0a6,code,# This Python 3 environment comes with many he...,0,0.000000,945aea18,None
1,1,00001756c60be8,448eb224,markdown,**Импортируем необходимые для работы функции и...,1,0.017241,945aea18,None
2,2,00001756c60be8,2a9e43d6,code,import numpy as np\nimport pandas as pd\nimpor...,2,0.034483,945aea18,None
3,3,00001756c60be8,7e2f170a,markdown,**Подключаем предупреждения**,3,0.051724,945aea18,None
4,4,00001756c60be8,038b763d,code,import warnings\nwarnings.filterwarnings('igno...,4,0.068966,945aea18,None


In [9]:
set_of_words = ['train', 'test', 'training', 'testing']

df['is_train_test'] = df.source.apply(lambda a: any(s in a for s in set_of_words))

In [10]:
df.is_train_test.sum() / df.is_train_test.size

0.2622558214661433

In [12]:
(df.groupby('cell_type').get_group('markdown').is_train_test.sum()
 / df.groupby('cell_type').get_group('markdown').is_train_test.size)


0.09490255135582328

In [16]:
grouped = df.groupby('cell_type').get_group('markdown').groupby('id')

In [18]:
ratio_by_groups = grouped.is_train_test.apply(lambda x: x.sum() / len(x))
ratio_by_groups.head()

id
00001756c60be8    0.0
00015c83e2717b    0.0
0001bdd4021779    0.0
0001daf4c2c76d    0.0
0002115f48f982    0.0
Name: is_train_test, dtype: float64

In [20]:
ratio_by_groups.idxmax()
grouped.get_group('001b382f2f0177')

'001b382f2f0177'

In [86]:
grouped_full = df.groupby('id')

In [87]:
from tqdm import tqdm
err = 0
def get_from_pairs(g):
    res = 0
    global err
    for _, md in g.groupby('cell_type').get_group('markdown').iterrows():
        nxt = g[g['rank'] == (md['rank']+1)]

        condition = (
                md['is_train_test'] == True
                and nxt.size
                and (nxt.cell_type.iloc[0] == 'code')
                and (nxt.is_train_test.iloc[0] == True)
        )
        if condition:
           res += 1
    return res / g.cell_id.size

tqdm.pandas()

r = grouped_full.progress_apply(lambda x: get_from_pairs(x))
r.head()


100%|██████████| 139256/139256 [08:06<00:00, 286.17it/s]


id
00001756c60be8    0.0
00015c83e2717b    0.0
0001bdd4021779    0.0
0001daf4c2c76d    0.0
0002115f48f982    0.0
dtype: float64

In [89]:
r.min(), r.mean(), r.max(), r.size

(0.0, 0.016840060853685116, 0.5, 139256)

In [91]:
r[r > 0].mean()

0.055975164440643876